In [1]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from datetime import datetime
from multiprocessing import Pool
from sklearn.svm import SVC

In [2]:
class ML:
    def __init__(self):
        self.data = pd.read_csv('mitbih_test.csv', header=None)
        self.fitness_history = []

    def loadData(self, selected_features):
        all_features = list(self.data.columns)
        selected_column_names = [all_features[i] for i in range(len(selected_features)) if selected_features[i] == 1]
        return self.data[selected_column_names].values, self.data.iloc[:, -1].values.astype(int)
    
    def obtenerCantidadColumnas(self):
        return len(self.data.columns)
    
    def fitness(self, features, X, y):
        selected_features = [i for i, val in enumerate(features) if val == 1]
        X_selected = X[:, selected_features]
        X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
        clf = SVC()
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        return accuracy_score(y_test, predictions)
        
    def parallel_fitness(self, nest, X, y):
        return self.fitness(nest, X, y)

    def levy_flight(self, beta):
        sigma = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) / 
                (math.gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
        u = np.random.normal(0, sigma, 1)
        v = np.random.normal(0, 1, 1)
        step = u / abs(v)**(1 / beta)
        return step

    def generate_nest(self, num_features):
        return np.random.choice([0, 1], size=num_features)
    
    def CSA(self, X, y, pa, max_iter, beta, num_nests=None):
        num_features = X.shape[1]

        if num_nests is None:
            num_nests = len(self.data.columns)
        
        nests = [self.generate_nest(num_features) for _ in range(num_nests)]
        with Pool(processes=3) as pool:  # Utiliza 4 procesos
            fitness_values = pool.starmap(self.parallel_fitness, [(nest, X, y) for nest in nests])
        best_nest = None
        best_score = -1

        for iter in range(max_iter):
            print(iter)
            scores = fitness_values.copy()
            self.fitness_history.append(max(scores))

            current_best_nest = nests[np.argmax(scores)]
            current_best_score = max(scores)

            if current_best_score > best_score:
                best_score = current_best_score
                best_nest = current_best_nest

            for i in range(num_nests):
                if np.random.rand() > pa:
                    for j in range(num_features):
                        if abs(self.levy_flight(beta)) > 0.5:
                            nests[i][j] = 1 - nests[i][j]
                    nests[i][np.random.choice(num_features)] = 1 - nests[i][np.random.choice(num_features)]
                    fitness_values[i] = self.fitness(nests[i], X, y)
        
        return best_nest, best_score

    def get_fitness_history(self):
        return self.fitness_history

In [3]:
ml_instance = ML()

X, y = ml_instance.loadData([1 for _ in range(ml_instance.obtenerCantidadColumnas()-1)]) # Cargar todas las características inicialmente


#Config: {'pa': 0.5, 'max_iter': 10, 'epsilon': 0.16, 'beta': 1.4, 'alpha': 0.1, 'gamma': 0.6}
config = "'pa': 0.5, 'max_iter': 100, 'epsilon': 0.16, 'beta': 1.4, 'alpha': 0.1, 'gamma': 0.6"
print("CONFIGURACION: ", config)
best_features, best_accuracy = ml_instance.CSA(X, y, pa=0.5, max_iter=100, beta=1.4)
fitness_evolution = ml_instance.get_fitness_history()
#Escribir los resultados en los archivos correspondientes
with open("RESULTADOS-CSA.txt", "a") as file:
    file.write("Fecha y hora: " + str(datetime.now()) + "\n")
    file.write("Parametros utilizados: " + str(config) + "\n")
    file.write("Mejores características: " + str(best_features) + "\n")
    file.write("Mejor precisión (fitness): " + str(best_accuracy) + "\n")
    file.write("-------------------------------------------------------------------------\n\n")
   
with open("FITNESSEVOLUTION-CSA.txt", "a") as file:
    file.write(str(datetime.now()))
    file.write("\n"+ "Evolucion del fitness: " + str(fitness_evolution) + "\n\n")
    file.write("-------------------------------------------------------------------------\n\n")

CONFIGURACION:  'pa': 0.5, 'max_iter': 100, 'epsilon': 0.16, 'beta': 1.4, 'alpha': 0.1, 'gamma': 0.6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
ml_instance = ML()

X, y = ml_instance.loadData([1 for _ in range(ml_instance.obtenerCantidadColumnas()-1)]) # Cargar todas las características inicialmente


parameter_space = {
    'pa': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95],
    'max_iter': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'beta': [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
}

def generate_initial_configurations(parameter_space, num_configurations):
    configurations = []
    for _ in range(num_configurations):
        config = {}
        for param, values in parameter_space.items():
            config[param] = np.random.choice(values)
        configurations.append(config)
    return configurations

def eliminate_worst_configurations(configurations, scores, elimination_rate=0.25):
    sorted_indices = np.argsort(scores)
    num_to_keep = max(1, int(len(configurations) * (1 - elimination_rate)))  # Guardar al menos 1
    return [configurations[i] for i in sorted_indices[:num_to_keep]]

num_configurations = 15  # o el número que prefieras
configurations = generate_initial_configurations(parameter_space, num_configurations)

# En este caso, simplemente vamos a correr I-Race por un número fijo de "epocas" o rondas
# También podrías implementar un criterio de convergencia o estancamiento si lo prefieres.
num_epochs = 5  

for epoch in range(num_epochs):
    scores = []
    
    for config in configurations:
        try:
            print("CONFIGURACION: ", config)
            best_features, score = ml_instance.CSA(X, y, **config)
            scores.append(score)
            fitness_evolution = ml_instance.get_fitness_history()
            #Escribir los resultados en los archivos correspondientes
            with open("RESULTADOS-CSA-IRACE.txt", "a") as file:
                file.write("Fecha y hora: " + str(datetime.now()) + "\n")
                file.write("Parametros utilizados: " + str(config) + "\n")
                file.write("Mejores características: " + str(best_features) + "\n")
                file.write("Mejor precisión (fitness): " + str(score) + "\n")
                file.write("-------------------------------------------------------------------------\n\n")
                        
            with open("FITNESSEVOLUTION-CSA-IRACE.txt", "a") as file:
                file.write(str(datetime.now()))
                file.write("\n"+ "Evolucion del fitness: " + str(fitness_evolution) + "\n\n")
                file.write("-------------------------------------------------------------------------\n\n")
                
        except Exception as e:  # Captura problemas en la ejecución del algoritmo
            print(f"Error with configuration {config}: {str(e)}")
            scores.append(np.inf)  # Puedes asignar un score infinito o manejar de otra manera.
    
    configurations = eliminate_worst_configurations(configurations, scores)

    # Logging
    if scores and configurations:
        print(f"Epoch {epoch + 1}/{num_epochs}, Best score: {scores[0]}, Config: {configurations[0]}")
    else:
        print(f"Epoch {epoch + 1}/{num_epochs}, No configurations or scores available")

# La mejor configuración se encuentra al principio de la lista
best_configuration = configurations[0]

print("Mejor configuracion encontrada:")
print(best_configuration)


CONFIGURACION:  {'pa': 0.9, 'max_iter': 30, 'beta': 1.8}
CONFIGURACION:  {'pa': 0.8, 'max_iter': 10, 'beta': 2.0}
CONFIGURACION:  {'pa': 0.7, 'max_iter': 40, 'beta': 1.8}
CONFIGURACION:  {'pa': 0.5, 'max_iter': 70, 'beta': 0.2}
CONFIGURACION:  {'pa': 0.6, 'max_iter': 90, 'beta': 1.8}
CONFIGURACION:  {'pa': 0.7, 'max_iter': 50, 'beta': 0.6}
CONFIGURACION:  {'pa': 0.6, 'max_iter': 40, 'beta': 0.4}
CONFIGURACION:  {'pa': 0.2, 'max_iter': 40, 'beta': 1.2}
CONFIGURACION:  {'pa': 0.3, 'max_iter': 20, 'beta': 0.4}
CONFIGURACION:  {'pa': 0.9, 'max_iter': 10, 'beta': 1.0}
CONFIGURACION:  {'pa': 0.8, 'max_iter': 40, 'beta': 1.0}
CONFIGURACION:  {'pa': 0.6, 'max_iter': 80, 'beta': 0.6}
CONFIGURACION:  {'pa': 0.95, 'max_iter': 10, 'beta': 1.6}
CONFIGURACION:  {'pa': 0.6, 'max_iter': 50, 'beta': 1.8}
CONFIGURACION:  {'pa': 0.6, 'max_iter': 80, 'beta': 0.4}
Epoch 1/5, Best score: 0.9684859556976478, Config: {'pa': 0.9, 'max_iter': 10, 'beta': 1.0}
CONFIGURACION:  {'pa': 0.9, 'max_iter': 10, 'beta': 